In [19]:
import numpy
import math
from matplotlib import pyplot

%matplotlib inline

In [9]:
N = 51
x_start, x_end = -2.0, 1.0
y_start, y_end = -0.5, 0.5
x = numpy.linspace(x_start,x_end,N)
y = numpy.linspace(y_start,y_end,N)
X, Y = numpy.meshgrid(x,y)

In [13]:
NACAx = numpy.loadtxt("NACA0012_x.txt")
NACAy = numpy.loadtxt("NACA0012_y.txt")
NACAsig = numpy.loadtxt("NACA0012_sigma.txt")

In [18]:
u_inf = 1.0

u_freestream = u_inf*numpy.ones((N,N),dtype=float)
v_freestream = numpy.zeros((N,N),dtype=float)

psi_freestream = u_inf*Y